In [30]:
import numpy as np
import pandas as pd
import igraph as ig

In [2]:
df = pd.DataFrame([
        {'player': 'alice', 'score': 10},
        {'player': 'bob', 'score': 7},
        {'player': 'charlie', 'score': 2},
        {'player': 'dennis', 'score': 5},
    ])

In [3]:
indices = np.triu_indices(len(df), k=1)

In [4]:
indices

(array([0, 0, 0, 1, 1, 2]), array([1, 2, 3, 2, 3, 3]))

In [5]:
sorted_df = df.sort_values('score', ascending=0)

In [6]:
sorted_df

,player,score
0,alice,10
1,bob,7
3,dennis,5
2,charlie,2


In [7]:
sorted_df.score.iloc[indices[0]]

0    10
0    10
0    10
1     7
1     7
3     5
Name: score, dtype: int64

In [8]:
sorted_df.score.iloc[indices[1]]

1    7
3    5
2    2
3    5
2    2
2    2
Name: score, dtype: int64

In [9]:
pairwise_diffs = sorted_df.score.values[indices[0]] - sorted_df.score.values[indices[1]]

In [10]:
pairwise_diffs

array([3, 5, 8, 2, 5, 3])

In [11]:
indices[0]

array([0, 0, 0, 1, 1, 2])

In [12]:
diffs = pd.DataFrame({
        'a': sorted_df.player.values[indices[0]],
        'b': sorted_df.player.values[indices[1]],
        'diff': pairwise_diffs
    })

In [13]:
diffs.set_index(['a', 'b'])

diff
a      b            
alice  bob         3
       dennis      5
       charlie     8
bob    dennis      2
       charlie     5
dennis charlie     3

In [14]:
diffs.pivot('a', 'b', 'diff').fillna(0)

b,bob,charlie,dennis
a,,,
alice,3.0,8.0,5.0
bob,0.0,5.0,2.0
dennis,0.0,3.0,0.0


In [15]:
df2 = pd.DataFrame([
        {'player': 'alice', 'score': 5},
        {'player': 'bob', 'score': 6},
        {'player': 'charlie', 'score': 3},
        {'player': 'dennis', 'score': 2},
    ])

In [16]:
sorted_df2 = df2.sort_values('score', ascending=0)
indices2 = np.triu_indices(len(df2), k=1)
diffs2 = pd.DataFrame({
        'a': sorted_df2.player.values[indices2[0]],
        'b': sorted_df2.player.values[indices2[1]],
        'diff': sorted_df2.score.values[indices2[0]] - sorted_df2.score.values[indices2[1]]
    })
assert diffs2.diff > 0

In [17]:
diffs2.pivot('a', 'b', 'diff').fillna(0)

b,alice,charlie,dennis
a,,,
alice,0.0,2.0,3.0
bob,1.0,3.0,4.0
charlie,0.0,0.0,1.0


In [18]:
diffs.pivot('a', 'b', 'diff').fillna(0) + diffs2.pivot('a', 'b', 'diff').fillna(0)

b,alice,bob,charlie,dennis
a,,,,
alice,NaN,NaN,10.0,8.0
bob,NaN,NaN,8.0,6.0
charlie,NaN,NaN,NaN,NaN
dennis,NaN,NaN,NaN,NaN


In [19]:
diffs.set_index(['a', 'b'])

diff
a      b            
alice  bob         3
       dennis      5
       charlie     8
bob    dennis      2
       charlie     5
dennis charlie     3

In [20]:
diffs2.set_index(['a', 'b'])

diff
a       b            
bob     alice       1
        charlie     3
        dennis      4
alice   charlie     2
        dennis      3
charlie dennis      1

In [22]:
df = diffs.set_index(['a', 'b']).add(diffs2.set_index(['a', 'b']), fill_value=0)

In [23]:
df

diff
a       b            
alice   bob       3.0
        charlie  10.0
        dennis    8.0
bob     alice     1.0
        charlie   8.0
        dennis    6.0
charlie dennis    1.0
dennis  charlie   3.0

In [29]:
df.reset_index().values

array([['alice', 'bob', 3.0],
       ['alice', 'charlie', 10.0],
       ['alice', 'dennis', 8.0],
       ['bob', 'alice', 1.0],
       ['bob', 'charlie', 8.0],
       ['bob', 'dennis', 6.0],
       ['charlie', 'dennis', 1.0],
       ['dennis', 'charlie', 3.0]], dtype=object)

In [32]:
g = ig.Graph.TupleList(df.reset_index().values, directed=True, edge_attrs=['weight'])

In [39]:
g.pagerank()

[0.05232558139534885,
 0.05232558139534885,
 0.4476744186046512,
 0.4476744186046512]